# MC_Issues framework demo

In [1]:
from test_issues import IntIssue, StringIssue
from random import choice
from string import ascii_uppercase

## use metric_type.calculate_all(payload) get all issues found with a payload

In [10]:
refs = []
for i in range(10):
    s = ''.join(choice(ascii_uppercase) for i in range(choice(list(range(5,15)))))
    r = StringIssue.calculate_all(s)
    refs.append(r)
    if len(r) != 0:
        print("\n".join(r_["template"] for r_ in r))
    else:
        print(f'No issues for {s}')
    print("#"*10)

VIXAQVSUMUKWQU is too long
VIXAQVSUMUKWQU contains duplicate letters
##########
WFDJDLYIHF contains duplicate letters
WFDJDLYIHF does not contain the letter 'a'
##########
TWMFJGXRDZSASH is too long
TWMFJGXRDZSASH contains duplicate letters
##########
NKVKSRTJEXJ is too long
NKVKSRTJEXJ contains duplicate letters
NKVKSRTJEXJ does not contain the letter 'a'
##########
SRWIOQBW is too short
SRWIOQBW contains duplicate letters
SRWIOQBW does not contain the letter 'a'
##########
CAIFQBQWVLUOUD is too long
CAIFQBQWVLUOUD contains duplicate letters
##########
YSVAGQUO is too short
##########
YARWMVQW is too short
YARWMVQW contains duplicate letters
##########
CIAQMWWEGN contains duplicate letters
##########
QMRTOU is too short
QMRTOU does not contain the letter 'a'
##########


## Use include_tags to only run on certain kinds of issue

In [3]:
for i in range(10):
    s = ''.join(choice(ascii_uppercase) for i in range(choice(list(range(5,15)))))
    r = StringIssue.calculate_all(s, include_tags=["letter"])
    if len(r) != 0:
        print("\n".join(r_["template"] for r_ in r))
    else:
        print(f'No issues for {s}')
    print("#"*10)

OCYCWNCX contains duplicate letters
OCYCWNCX does not contain the letter 'a'
##########
MKQLS does not contain the letter 'a'
##########
LLGUJZOGQ contains duplicate letters
LLGUJZOGQ does not contain the letter 'a'
##########
ZCGIZSFD contains duplicate letters
ZCGIZSFD does not contain the letter 'a'
##########
XNZNC contains duplicate letters
XNZNC does not contain the letter 'a'
##########
DJSSPBMUJSWOV contains duplicate letters
DJSSPBMUJSWOV does not contain the letter 'a'
##########
RWFOVYWXKP contains duplicate letters
RWFOVYWXKP does not contain the letter 'a'
##########
MZELWGJKKJYD contains duplicate letters
MZELWGJKKJYD does not contain the letter 'a'
##########
TQLYTDOXDPIBW contains duplicate letters
TQLYTDOXDPIBW does not contain the letter 'a'
##########
KKAVVYOGJVRHNG contains duplicate letters
##########


## And use exclude_tags to limit what kinds of issues you're checking for

In [4]:
for i in range(10):
    s = ''.join(choice(ascii_uppercase) for i in range(choice(list(range(5,15)))))
    r = StringIssue.calculate_all(s, exclude_tags=["letter"])
    if len(r) != 0:
        print("\n".join(r_["template"] for r_ in r))
    else:
        print(f'No issues for {s}')
    print("#"*10)

MVZZRPN is too short
##########
AZLOPWRDZ is too short
##########
GJWGJFV is too short
##########
FWYQPR is too short
##########
FPGAXRUHUEZTI is too long
##########
JQUARU is too short
##########
No issues for LLRGRACHNX
##########
UARQAZAAR is too short
##########
XZSDUHJ is too short
##########
VEHYVKBTH is too short
##########


## The framework tries to catch and report errors in issue calculation gracefully
Note that execution isn't halted- we just log an error, and report that the issue didn't calculate right. 

In [5]:
print(IntIssue.calculate_all(2, exclude_tags=["breaking"]))
print(IntIssue.calculate_all(3))

ERROR:issues:Error calculating issue broken_issue for payload 3: Intentional Error
Traceback (most recent call last):
  File "/Users/pgulley/Projects/directory-issues/issues.py", line 62, in calculate_all
    is_issue, result_data = issue_instance.calculate(payload)
                            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/pgulley/Projects/directory-issues/test_issues.py", line 63, in calculate
    raise RuntimeError("Intentional Error")
RuntimeError: Intentional Error


[{'issue_name': 'is_even', 'template': '2 is even', 'tags': []}]
[{'issue_name': 'broken_issue', 'tags': ['breaking'], 'error': True, 'error_message': 'Intentional Error', 'template': "An error occurred while calculating 'broken_issue'"}]


# Usage:

We shouldn't need very many extensions on the base issue class- we'll certainly need a SourceIssue, FeedIssue, and a CollectionIssue but that's probably it. 
Regardless, generating one is a two-liner: 

In [6]:
from issues import IssueBase
from typing import TypeVar, Tuple, Generic, Type, Dict, Callable, List, Any

class FloatIssue(IssueBase[float]):
    _ISSUES: Dict[str, Type["FloatIssue"]] = {}


Then actually implimenting the issue cases requires creating a class with just two methods- `calculate(payload)` and `render_template()`, then decorating it correctly:

In [7]:
@FloatIssue.register("decimal_places", tags=['decimal'])
class DecimalPlaces(FloatIssue):
    def calculate(self, payload:float) -> Tuple[bool, Dict]:        
        """
        NB the return signature: Tuple[bool, Any]
        bool is ultimately interpreted as "is_issue" - if false, the result is skipped
        Dict is the input provided to the render template
        """
        num, dec = str(payload).split(".")
        exp = len(dec)
    
        return exp > 2, {'value':payload, "exp":exp}

    def render_template(self):
        return f"{self.result['value']} has too many decimal places ({self.result['exp']})"


@FloatIssue.register("negative")
class NegativeFloat(FloatIssue):
    def calculate(self, payload:float):
        return payload < 0, {'value':payload}

    def render_template(self):
        return f"{self.result['value']} is negative"
            

In [8]:
for i in (1.2, -4.7 ,1.24525):
    r = FloatIssue.calculate_all(i)
    print(r)

[]
[{'issue_name': 'negative', 'template': '-4.7 is negative', 'tags': []}]
[{'issue_name': 'decimal_places', 'template': '1.24525 has too many decimal places (5)', 'tags': ['decimal']}]


At dev time, you should only be calling an issue from the calculate_all method- there's no need to ever call issue.calculate or issue.render_template directly. 


In [13]:
print(refs[3])

[{'issue_name': 'long_str', 'template': 'NKVKSRTJEXJ is too long', 'tags': []}, {'issue_name': 'dup_letter', 'template': 'NKVKSRTJEXJ contains duplicate letters', 'tags': ['letter']}, {'issue_name': 'no_a', 'template': "NKVKSRTJEXJ does not contain the letter 'a'", 'tags': ['letter']}]


In [14]:
from collections import defaultdict

def group_issues(issues):
    grouped_issues = defaultdict(list)
    for issue in issues:
        primary_tag = issue['tags'][0] if issue['tags'] else 'no_tag'
        grouped_issues[primary_tag].append(issue['template'])
    return grouped_issues

In [17]:
t = group_issues(refs[3])

In [18]:
t["no_tag"]

['NKVKSRTJEXJ is too long']

In [19]:
t

defaultdict(list,
            {'no_tag': ['NKVKSRTJEXJ is too long'],
             'letter': ['NKVKSRTJEXJ contains duplicate letters',
              "NKVKSRTJEXJ does not contain the letter 'a'"]})

In [20]:
if not None: print('hi')

hi
